In [1]:
import matplotlib
matplotlib.use('Agg')  #Fixing AWS Lambda MPL backend issue

import matplotlib.pyplot as plt
import matplotlib.dates as md
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
from datetime import datetime,date,timedelta
import boto3
import pytz 
from bs4 import BeautifulSoup
import requests
import io

params = {
'token' : 'jgyuNE4NXwPdjzmQS568me3TyXLAL9ZCU9NkVlKj'
,'bot_id' :'4e93908dd6e03b66cbd07fc458' #Test
}

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('MatchupGameFlow')

leagueId = '111414'
season = datetime.now().astimezone(pytz.timezone('US/Eastern')).year

r = requests.get('http://www.nfl.com/liveupdate/scorestrip/ss.xml')
schedxml = BeautifulSoup(r.text, "html.parser")
#week = schedxml.find('gms')['w']
week = 'P1'

def _is_nfl_game_active():
    r = requests.get('http://www.nfl.com/liveupdate/scorestrip/ss.xml')
    schedxml = BeautifulSoup(r.text, "html.parser")
    active = False
    for info in schedxml.findAll('g'):        
        gamestatus = info['q']
        if(gamestatus != 'F'  and gamestatus != 'FO' and gamestatus != 'P'):
            active = True  
            break
        
    return active # return active

#_is_nfl_game_active()

In [2]:
def _last_game_ended_recently():
    r = requests.get('http://www.nfl.com/liveupdate/scorestrip/ss.xml')
    schedxml = BeautifulSoup(r.text, "html.parser")

#    current_utc_ts = pytz.utc.localize(datetime(2018, 11, 5, 18, 59))
    current_utc_ts = pytz.utc.localize(datetime.utcnow())
    last_game_end_utc_ts = pytz.utc.localize(datetime(2000, 1, 1, 0, 0))
    local_tz = pytz.timezone('US/Eastern')

    for info in schedxml.findAll('g'):
        hour, minute = info['t'].strip().split(':')
        d = local_tz.localize(datetime(int(info['eid'][:4]), int(info['eid'][4:6]), int(info['eid'][6:8]),
                              (int(hour) + 12) % 24, int(minute))) 
        print(d)
        if(d.hour == 21 and d.minute == 30): #Saturday 9:30am games need to be adjusted from PM to AM
            d = d - timedelta(hours=12)
        
        
        start_utc = d.astimezone(pytz.timezone('UTC')) #convert to UTC for AWS Lambda environment
        end_utc = d.astimezone(pytz.timezone('UTC')) + timedelta(hours=4.5) #convert to UTC for AWS Lambda environment

        #print(current_utc_ts,start_utc,end_utc,current_utc_ts >= start_utc and current_utc_ts <= end_utc )
        if(current_utc_ts >= start_utc and current_utc_ts <= end_utc):
            return True        

    return False

_last_game_ended_recently()


2018-12-13 20:20:00-05:00
2018-12-15 16:30:00-05:00
2018-12-15 20:20:00-05:00
2018-12-16 13:00:00-05:00
2018-12-16 13:00:00-05:00
2018-12-16 13:00:00-05:00
2018-12-16 13:00:00-05:00
2018-12-16 13:00:00-05:00
2018-12-16 13:00:00-05:00
2018-12-16 13:00:00-05:00
2018-12-16 13:00:00-05:00
2018-12-16 13:00:00-05:00
2018-12-16 16:05:00-05:00
2018-12-16 16:25:00-05:00
2018-12-16 20:20:00-05:00
2018-12-17 20:15:00-05:00


False

In [3]:
def generate_charts():
    data = table.scan()

    df = json_normalize(data, 'Items')[['COLLECTTIMESTAMP','TEAM1','TEAM1NAME','TEAM1PTS','TEAM1PROJ','TEAM2NAME','TEAM2PTS','TEAM2PROJ','SCORINGPERIOD']]
    df = df[df['SCORINGPERIOD']==week].sort_values('COLLECTTIMESTAMP', ascending=True)
    #df[['COLLECTTIMESTAMP']] = df[['COLLECTTIMESTAMP']].apply(pd.to_datetime)  # commenting removes attempt at date labels on x-axis
    df[['TEAM1PTS','TEAM1PROJ','TEAM2PTS','TEAM2PROJ']] = df[['TEAM1PTS','TEAM1PROJ','TEAM2PTS','TEAM2PROJ']].apply(pd.to_numeric)


    imgs = []
    for home_team in df['TEAM1NAME'].unique():
        matchup = df[df['TEAM1NAME']==home_team]
        team1 = matchup['TEAM1NAME'].unique()[0]
        team2 = matchup['TEAM2NAME'].unique()[0]

        fig, ax = plt.subplots(1,1)

        plt.title(team1+' vs. '+team2+'\nWeek '+week)
        #ax.xaxis.set_minor_locator(md.HourLocator(interval=4))   # every 4 hours
        #ax.xaxis.set_minor_formatter(md.DateFormatter('%H:%M'))  # hours and minutes
        #ax.xaxis.set_major_locator(md.HourLocator(interval=12))    # every day
        #ax.xaxis.set_major_formatter(md.DateFormatter('\n%a'))
        #ax.set_xlim(datetime(2018, 10, 1, 20), datetime(2018, 10, 2 , 0))
        ax.xaxis.set_major_locator(plt.NullLocator())
        
        line_t1pts, = ax.plot(matchup['COLLECTTIMESTAMP'], matchup['TEAM1PTS'], 'r', label=matchup['TEAM1NAME'].unique()[0]+' PTS')
        line_t1proj, = ax.plot(matchup['COLLECTTIMESTAMP'], matchup['TEAM1PROJ'], 'r',linestyle='--', dashes=(2, 2), label=matchup['TEAM1NAME'].unique()[0]+' PROJ')
        line_t2pts, = ax.plot(matchup['COLLECTTIMESTAMP'], matchup['TEAM2PTS'], 'b', label=matchup['TEAM2NAME'].unique()[0]+' PTS')
        line_t2proj, = ax.plot(matchup['COLLECTTIMESTAMP'], matchup['TEAM2PROJ'], 'b',linestyle='--', dashes=(2, 2), label=matchup['TEAM2NAME'].unique()[0]+' PROJ')
        ax.legend(loc='best')

        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        imgs.append(buf)
        #plt.show()
        plt.close()
    return imgs



In [4]:
def post_to_groupme(input, imgs):
    headers_img = {
        'X-Access-Token': input['token'],
        'Content-Type': 'image/jpeg',
    }

    headers_post = {
        'Content-Type': 'application/json',
    }

    for img in imgs:
        img.seek(0)
        img = requests.post('https://image.groupme.com/pictures', headers=headers_img, data=img)
        imgurl = img.json()['payload']['picture_url']
        data = '{"bot_id":"'+input['bot_id']+'","text":"","attachments":[{"type":"image","url":"'+imgurl+'"}]}'
        response = requests.post('https://api.groupme.com/v3/bots/post', headers=headers_post, data=data)



In [5]:
def lambda_handler(input,context):
    if not _is_nfl_game_active() and not _last_game_ended_recently():
        return False
    
    post_to_groupme(input, generate_charts())
    return True



In [13]:
def get_data():
    data = table.scan()

    df = json_normalize(data, 'Items')
#    df = json_normalize(data, 'Items')[['COLLECTTIMESTAMP','TEAM1','TEAM1NAME','TEAM1PTS','TEAM1PROJ','TEAM2NAME','TEAM2PTS','TEAM2PROJ','SCORINGPERIOD']]
    df = df[df['SCORINGPERIOD']==week].sort_values('COLLECTTIMESTAMP', ascending=True)
    #df[['COLLECTTIMESTAMP']] = df[['COLLECTTIMESTAMP']].apply(pd.to_datetime)  # commenting removes attempt at date labels on x-axis
#    df[['TEAM1PTS','TEAM1PROJ','TEAM2PTS','TEAM2PROJ']] = df[['TEAM1PTS','TEAM1PROJ','TEAM2PTS','TEAM2PROJ']].apply(pd.to_numeric)
    return data

data = get_data()

#df[df['TEAM1NAME']=='Paul'].sort_values('COLLECTTIMESTAMP',ascending=False)




In [64]:
for item in data['Items']:
    if item['WEEK_NM'] == 'P1':
        print(item['SCORINGPERIOD'], item['COLLECTTIMESTAMP'], item['TEAM1'], item['WEEK_NM'])

        response = table.update_item(
            Key={
                'COLLECTTIMESTAMP': item['COLLECTTIMESTAMP'],
                'TEAM1': item['TEAM1']
            },
            UpdateExpression="set SCORINGPERIOD = :s",
            ExpressionAttributeValues={
                ':s': "15"
            },
            ReturnValues="UPDATED_NEW"
        )

15 2018-12-13 21:43:31 11 P1
15 2018-12-13 21:43:31 12 P1
15 2018-12-13 21:43:31 3 P1
15 2018-12-13 21:43:31 4 P1
15 2018-12-13 21:43:31 7 P1
15 2018-12-13 21:43:31 8 P1
15 2018-12-13 22:03:31 11 P1
15 2018-12-13 22:03:31 12 P1
15 2018-12-13 22:03:31 3 P1
15 2018-12-13 22:03:31 4 P1
15 2018-12-13 22:03:31 7 P1
15 2018-12-13 22:03:31 8 P1
15 2018-12-13 20:44:31 11 P1
15 2018-12-13 20:44:31 12 P1
15 2018-12-13 20:44:31 3 P1
15 2018-12-13 20:44:31 4 P1
15 2018-12-13 20:44:31 7 P1
15 2018-12-13 20:44:31 8 P1
15 2018-12-13 22:37:31 11 P1
15 2018-12-13 22:37:31 12 P1
15 2018-12-13 22:37:31 3 P1
15 2018-12-13 22:37:31 4 P1
15 2018-12-13 22:37:31 7 P1
15 2018-12-13 22:37:31 8 P1
15 2018-12-13 22:01:31 11 P1
15 2018-12-13 22:01:31 12 P1
15 2018-12-13 22:01:31 3 P1
15 2018-12-13 22:01:31 4 P1
15 2018-12-13 22:01:31 7 P1
15 2018-12-13 22:01:31 8 P1
15 2018-12-13 23:15:31 11 P1
15 2018-12-13 23:15:31 12 P1
15 2018-12-13 23:15:31 3 P1
15 2018-12-13 23:15:31 4 P1
15 2018-12-13 23:15:31 7 P1
15 2018-

In [30]:
data = get_data()


In [69]:
for item in data['Items']:
    #print(item['SCORINGPERIOD'])
#    if item['SCORINGPERIOD'] == '15':
    if item['WEEK_NM'] == 'P1':
        print(item['SCORINGPERIOD'], item['COLLECTTIMESTAMP'], item['TEAM1'], item['WEEK_NM'])


    

15 2018-12-13 21:43:31 11 P1
15 2018-12-13 21:43:31 12 P1
15 2018-12-13 21:43:31 3 P1
15 2018-12-13 21:43:31 4 P1
15 2018-12-13 21:43:31 7 P1
15 2018-12-13 21:43:31 8 P1
15 2018-12-13 22:03:31 11 P1
15 2018-12-13 22:03:31 12 P1
15 2018-12-13 22:03:31 3 P1
15 2018-12-13 22:03:31 4 P1
15 2018-12-13 22:03:31 7 P1
15 2018-12-13 22:03:31 8 P1
15 2018-12-13 20:44:31 11 P1
15 2018-12-13 20:44:31 12 P1
15 2018-12-13 20:44:31 3 P1
15 2018-12-13 20:44:31 4 P1
15 2018-12-13 20:44:31 7 P1
15 2018-12-13 20:44:31 8 P1
15 2018-12-13 22:37:31 11 P1
15 2018-12-13 22:37:31 12 P1
15 2018-12-13 22:37:31 3 P1
15 2018-12-13 22:37:31 4 P1
15 2018-12-13 22:37:31 7 P1
15 2018-12-13 22:37:31 8 P1
15 2018-12-13 22:01:31 11 P1
15 2018-12-13 22:01:31 12 P1
15 2018-12-13 22:01:31 3 P1
15 2018-12-13 22:01:31 4 P1
15 2018-12-13 22:01:31 7 P1
15 2018-12-13 22:01:31 8 P1
15 2018-12-13 23:15:31 11 P1
15 2018-12-13 23:15:31 12 P1
15 2018-12-13 23:15:31 3 P1
15 2018-12-13 23:15:31 4 P1
15 2018-12-13 23:15:31 7 P1
15 2018-